# Tully-Fisher Secondary Target Sample: Coma Cluster

Access secondary PV survey targets intended for TF measurements.

Grab the targets from the Coma cluster and see if we can calibrate the TFR.

In [1]:
import os
import requests

from astropy.io import ascii, fits
from astropy.table import join, hstack, vstack, unique, Table
from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy import units as u
from astropy.wcs import WCS
from astropy.visualization.wcsaxes import SphericalCircle

from desispec.spectra import stack as specstack
from desispec.io import read_spectra, write_spectra

import psycopg2

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'desispec'